In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import Input, LocallyConnected1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 300
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 300
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 120, 32

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
# import os
path='/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/sub_wise_process_TT/'

In [15]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],300,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [16]:
n_length =  300
n_depth=8
n_channel=8
n_outputs=26

In [17]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,8)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model   

In [18]:
if __name__ == "__main__":
    model = generate_model()

(None, 300, 8)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
ip_shape (None, 300)
ip_shape (None, 300, 1)
conv1_o (None, 300, 64)
(None, 300, 8, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 153600)
(None, 26)
inputs_shape (None, 300, 8)


In [19]:
for i in range(1,18):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test).;
    df_Test.head(5)
#     feature standardization
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_length =  300
    n_depth=8
    n_channel=8
    x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5'
#     model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    test_mean=statistics.mean(test_acc)
    print('test_mean for %d subjects:',i)
    print(test_mean)

/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 312000

/media/naveen/nav/mat_codes/BioPatRec/prep_LP_wave/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 156000

x_train shape:  (1039, 300, 8)
1039 training samples
y_train shape:  (1039,)
num_time_periods 300
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0]
x_train shape: (300, 8)
input_shape: (300, 8)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (1039, 26)
x_test shape:  (519, 300, 8)
519 testing samples
y_test shape:  (519,)
x_train shape:  (1039, 300, 8)
x_test shape:  (519, 300, 8)
n_outputs 26
Model: "wei_mscnn_model"
_______

33/33 [==============================] - ETA: 0s - loss: 4.7379 - accuracy: 0.1396
Epoch 00001: val_accuracy improved from -inf to 0.08863, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
33/33 [==============================] - 50s 2s/step - loss: 4.7379 - accuracy: 0.1396 - val_loss: 4.8373 - val_accuracy: 0.0886 - lr: 0.0010
Epoch 2/120
33/33 [==============================] - ETA: 0s - loss: 3.9889 - accuracy: 0.3359
Epoch 00002: val_accuracy improved from 0.08863 to 0.42197, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
33/33 [==============================] - 8s 243ms/step - loss: 3.9889 - accuracy: 0.3359 - val_loss: 3.7297 - val_accuracy: 0.4220 - lr: 0.0010
Epoch 3/120
33/33 [==============================] - ETA: 0s - loss: 3.3411 - accuracy: 0.4629
Epoch 00003: val_accuracy improved from 0.42197 to 0.49711, saving model to /media/naveen/nav/mat_codes/nina_DB1

Epoch 27/120
33/33 [==============================] - ETA: 0s - loss: 0.9722 - accuracy: 0.8720
Epoch 00027: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 161ms/step - loss: 0.9722 - accuracy: 0.8720 - val_loss: 2.1121 - val_accuracy: 0.5299 - lr: 0.0010
Epoch 28/120
33/33 [==============================] - ETA: 0s - loss: 0.9559 - accuracy: 0.8643
Epoch 00028: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 159ms/step - loss: 0.9559 - accuracy: 0.8643 - val_loss: 1.9396 - val_accuracy: 0.5800 - lr: 0.0010
Epoch 29/120
33/33 [==============================] - ETA: 0s - loss: 0.9300 - accuracy: 0.8778
Epoch 00029: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 158ms/step - loss: 0.9300 - accuracy: 0.8778 - val_loss: 1.8920 - val_accuracy: 0.5934 - lr: 0.0010
Epoch 30/120
33/33 [==============================] - ETA: 0s - loss: 0.9381 - accuracy: 0.8730
Epoch 00030: val_ac

Epoch 55/120
33/33 [==============================] - ETA: 0s - loss: 0.6522 - accuracy: 0.9278
Epoch 00055: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 160ms/step - loss: 0.6522 - accuracy: 0.9278 - val_loss: 2.0128 - val_accuracy: 0.5568 - lr: 1.0000e-04
Epoch 56/120
33/33 [==============================] - ETA: 0s - loss: 0.6709 - accuracy: 0.9124
Epoch 00056: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 162ms/step - loss: 0.6709 - accuracy: 0.9124 - val_loss: 1.9646 - val_accuracy: 0.5723 - lr: 1.0000e-04
Epoch 57/120
33/33 [==============================] - ETA: 0s - loss: 0.6520 - accuracy: 0.9220
Epoch 00057: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 158ms/step - loss: 0.6520 - accuracy: 0.9220 - val_loss: 1.9687 - val_accuracy: 0.5761 - lr: 1.0000e-04
Epoch 58/120
33/33 [==============================] - ETA: 0s - loss: 0.6591 - accuracy: 0.9192
Epoch 0

Epoch 83/120
33/33 [==============================] - ETA: 0s - loss: 0.5526 - accuracy: 0.9278
Epoch 00083: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 162ms/step - loss: 0.5526 - accuracy: 0.9278 - val_loss: 1.9845 - val_accuracy: 0.5645 - lr: 1.0000e-05
Epoch 84/120
33/33 [==============================] - ETA: 0s - loss: 0.5580 - accuracy: 0.9288
Epoch 00084: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 159ms/step - loss: 0.5580 - accuracy: 0.9288 - val_loss: 1.9656 - val_accuracy: 0.5588 - lr: 1.0000e-05
Epoch 85/120
33/33 [==============================] - ETA: 0s - loss: 0.5577 - accuracy: 0.9249
Epoch 00085: val_accuracy did not improve from 0.62813
33/33 [==============================] - 5s 161ms/step - loss: 0.5577 - accuracy: 0.9249 - val_loss: 1.9726 - val_accuracy: 0.5723 - lr: 1.0000e-05
Epoch 86/120
33/33 [==============================] - ETA: 0s - loss: 0.5623 - accuracy: 0.9230
Epoch 0

...

Epoch 84/120
33/33 [==============================] - ETA: 0s - loss: 0.5753 - accuracy: 0.9326
Epoch 00084: val_accuracy did not improve from 0.52216
33/33 [==============================] - 5s 160ms/step - loss: 0.5753 - accuracy: 0.9326 - val_loss: 2.6189 - val_accuracy: 0.4566 - lr: 1.0000e-05
Epoch 85/120
33/33 [==============================] - ETA: 0s - loss: 0.5647 - accuracy: 0.9336
Epoch 00085: val_accuracy did not improve from 0.52216
33/33 [==============================] - 5s 159ms/step - loss: 0.5647 - accuracy: 0.9336 - val_loss: 2.6321 - val_accuracy: 0.4682 - lr: 1.0000e-05
Epoch 86/120
33/33 [==============================] - ETA: 0s - loss: 0.5615 - accuracy: 0.9355
Epoch 00086: val_accuracy did not improve from 0.52216
33/33 [==============================] - 5s 159ms/step - loss: 0.5615 - accuracy: 0.9355 - val_loss: 2.6988 - val_accuracy: 0.4316 - lr: 1.0000e-05
Epoch 87/120
33/33 [==============================] - ETA: 0s - loss: 0.5470 - accuracy: 0.9413
Epoch 0

33/33 [==============================] - ETA: 0s - loss: 3.1808 - accuracy: 0.2868
Epoch 00001: val_accuracy improved from -inf to 0.38728, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
33/33 [==============================] - 33s 994ms/step - loss: 3.1808 - accuracy: 0.2868 - val_loss: 2.5150 - val_accuracy: 0.3873 - lr: 0.0010
Epoch 2/120
33/33 [==============================] - ETA: 0s - loss: 2.2397 - accuracy: 0.4937
Epoch 00002: val_accuracy improved from 0.38728 to 0.43160, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_BPR_20X10/checkpoint.hdf5
33/33 [==============================] - 8s 256ms/step - loss: 2.2397 - accuracy: 0.4937 - val_loss: 2.2750 - val_accuracy: 0.4316 - lr: 0.0010
Epoch 3/120
33/33 [==============================] - ETA: 0s - loss: 1.8133 - accuracy: 0.6160
Epoch 00003: val_accuracy improved from 0.43160 to 0.46050, saving model to /media/naveen/nav/mat_codes/nina_

Epoch 28/120
33/33 [==============================] - ETA: 0s - loss: 0.9410 - accuracy: 0.8932
Epoch 00028: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 154ms/step - loss: 0.9410 - accuracy: 0.8932 - val_loss: 2.9882 - val_accuracy: 0.4451 - lr: 0.0010
Epoch 29/120
33/33 [==============================] - ETA: 0s - loss: 0.9543 - accuracy: 0.8826
Epoch 00029: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 156ms/step - loss: 0.9543 - accuracy: 0.8826 - val_loss: 3.1156 - val_accuracy: 0.4547 - lr: 0.0010
Epoch 30/120
33/33 [==============================] - ETA: 0s - loss: 0.9509 - accuracy: 0.8826
Epoch 00030: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 157ms/step - loss: 0.9509 - accuracy: 0.8826 - val_loss: 3.3708 - val_accuracy: 0.4066 - lr: 0.0010
Epoch 31/120
33/33 [==============================] - ETA: 0s - loss: 1.0164 - accuracy: 0.8701
Epoch 00031: val_ac

Epoch 56/120
33/33 [==============================] - ETA: 0s - loss: 0.5757 - accuracy: 0.9471
Epoch 00056: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 156ms/step - loss: 0.5757 - accuracy: 0.9471 - val_loss: 3.0452 - val_accuracy: 0.4412 - lr: 1.0000e-04
Epoch 57/120
33/33 [==============================] - ETA: 0s - loss: 0.5807 - accuracy: 0.9451
Epoch 00057: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 156ms/step - loss: 0.5807 - accuracy: 0.9451 - val_loss: 3.0307 - val_accuracy: 0.4451 - lr: 1.0000e-04
Epoch 58/120
33/33 [==============================] - ETA: 0s - loss: 0.5509 - accuracy: 0.9596
Epoch 00058: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 158ms/step - loss: 0.5509 - accuracy: 0.9596 - val_loss: 3.0600 - val_accuracy: 0.4393 - lr: 1.0000e-04
Epoch 59/120
33/33 [==============================] - ETA: 0s - loss: 0.5406 - accuracy: 0.9538
Epoch 0

Epoch 84/120
33/33 [==============================] - ETA: 0s - loss: 0.4713 - accuracy: 0.9519
Epoch 00084: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 158ms/step - loss: 0.4713 - accuracy: 0.9519 - val_loss: 3.0133 - val_accuracy: 0.4547 - lr: 1.0000e-05
Epoch 85/120
33/33 [==============================] - ETA: 0s - loss: 0.4182 - accuracy: 0.9702
Epoch 00085: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 155ms/step - loss: 0.4182 - accuracy: 0.9702 - val_loss: 3.0473 - val_accuracy: 0.4528 - lr: 1.0000e-05
Epoch 86/120
33/33 [==============================] - ETA: 0s - loss: 0.4364 - accuracy: 0.9692
Epoch 00086: val_accuracy did not improve from 0.46628
33/33 [==============================] - 5s 157ms/step - loss: 0.4364 - accuracy: 0.9692 - val_loss: 3.0533 - val_accuracy: 0.4528 - lr: 1.0000e-05
Epoch 87/120
33/33 [==============================] - ETA: 0s - loss: 0.4252 - accuracy: 0.9682
Epoch 0

In [20]:
statistics.mean(test_acc)

0.6618497092276812